# Cassandra
Assumptions:
- Docker installed on system
- "cassandra:latest" image installed in docker
- Python/Conda environment with python 3.8 (or newer*)  
  
*As of 9 August 2023, the officially built cassandra-driver package was at version 3.25 and did not work well with Python 3.10 on Mac.  
To install a newer version from the terminal you can:  
```
pip install git+https://github.com/datastax/python-driver.git
pip install cql
```
  
Or use older Python:  
```
conda create --name cassy38 python=3.8 jupyter scikit-learn
conda activate cassy38  
pip install cassandra-driver cql
```

## Spinning up a local cassandra instance in a terminal
```docker run --name my_cassandra cassandra:latest -p 9042:9042```  
  
If this works for you, hourra! Otherwise, try the following:  
- Open Docker Desktop
- Run the cassandra container with optional settings, opening 9042 port (left-hand side).

In [1]:
# Connecting to Cassandra
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

In [2]:
# Set up new keyspace (first time only)
#                                              name of keyspace                        replication strategy           replication factor
session.execute("CREATE KEYSPACE IF NOT EXISTS my_first_keyspace WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")

In [3]:
# Create a new table (first time only)
session.set_keyspace('my_first_keyspace')
session.execute("CREATE TABLE IF NOT EXISTS my_first_table (ind int PRIMARY KEY, company text, model text);")

In [4]:
# Insert some data (ind is the primary key, must be unique)
session.execute("INSERT INTO my_first_table (ind, company, model) VALUES (1, 'Tesla', 'Model S');")
session.execute("INSERT INTO my_first_table (ind, company, model) VALUES (2, 'Tesla', 'Model 3');")
session.execute("INSERT INTO my_first_table (ind, company, model) VALUES (3, 'Polestar', '3');")

In [5]:
# Query the data
rows = session.execute("SELECT * FROM my_first_table;")
for i in rows:
    print(i)

Row(ind=1, company='Tesla', model='Model S')
Row(ind=2, company='Tesla', model='Model 3')
Row(ind=3, company='Polestar', model='3')


In [6]:
# More specific query
prepared_statement = session.prepare("SELECT * FROM my_first_table WHERE company=?;") # <- will fail as company is not a key
teslas = session.execute(prepared_statement, ['Tesla'])
for i in teslas:
    print(i)

InvalidRequest: Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"

## Cassandra filtering
Cassandra is inherently a distributed production database. Selecting as above may require downloading all data from a node, then filtering based on the WHERE part (only PRIMARY KEYs are centrally known). Solutions:  
- If the table is small or most of the data will satisfy the query, add ```ALLOW FILTERING``` at the end of the query (not recommended if not known). 
- Or make sure the WHERE clause points to one of the keys (see below).

In [7]:
# Create a new table (observe keys)
session.execute("CREATE TABLE IF NOT EXISTS car_table (company text, model text, PRIMARY KEY(company, model));")

In [8]:
# Insert some data (combination of company and model must be unique)
session.execute("INSERT INTO car_table (company, model) VALUES ('Tesla', 'Model S');")
session.execute("INSERT INTO car_table (company, model) VALUES ('Tesla', 'Model 3');")
session.execute("INSERT INTO car_table (company, model) VALUES ('Polestar', '3');")

In [9]:
# More specific query now works
prepared_statement = session.prepare("SELECT * FROM car_table WHERE company=?;")
teslas = session.execute(prepared_statement, ['Tesla'])
for i in teslas:
    print(i)

Row(company='Tesla', model='Model 3')
Row(company='Tesla', model='Model S')


## JSON in Cassandra


In [10]:
# Imports
import datetime as dt
import requests
import json

### Read previously saved JSON file forecast.json to memory

In [11]:
with open('../APIs/downloads/forecast.json', 'r') as f:
    forecast = json.load(f)

In [12]:
# Inspect JSON file
forecast.__str__()

"{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1691582400, 'main': {'temp': 286.26, 'feels_like': 285.95, 'temp_min': 283.99, 'temp_max': 286.26, 'pressure': 996, 'sea_level': 996, 'grnd_level': 982, 'humidity': 89, 'temp_kf': 2.27}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'clouds': {'all': 100}, 'wind': {'speed': 4.25, 'deg': 39, 'gust': 9.51}, 'visibility': 10000, 'pop': 0.91, 'rain': {'3h': 0.77}, 'sys': {'pod': 'd'}, 'dt_txt': '2023-08-09 12:00:00'}, {'dt': 1691593200, 'main': {'temp': 285.4, 'feels_like': 285.06, 'temp_min': 284.4, 'temp_max': 285.4, 'pressure': 999, 'sea_level': 999, 'grnd_level': 984, 'humidity': 91, 'temp_kf': 1}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'clouds': {'all': 100}, 'wind': {'speed': 3.07, 'deg': 4, 'gust': 6.57}, 'visibility': 10000, 'pop': 0.51, 'sys': {'pod': 'd'}, 'dt_txt': '2023-08-09 15:00:00'}, {'dt': 1691604000, 'main': {'temp': 284.13

## Raw JSON
A simple, but not very efficient way of storing JSON data is to treat it as a text and save it directly to the database.  
More efficient, with regard to transfer, is to compress the JSON data to a blob first.

In [13]:
session.execute("DROP TABLE IF EXISTS my_first_keyspace.forecast_table;")

### Create a new table which treats the whole JSON as a blob, using the city id and the first dt as keys

In [14]:
session.set_keyspace('my_first_keyspace')
session.execute("CREATE TABLE IF NOT EXISTS forecast_table (city_id int, dt int, forecast blob, PRIMARY KEY(city_id, dt));")

### Insert the forecast data into the table as text

In [15]:
session.execute("INSERT INTO forecast_table (city_id, dt, forecast) VALUES (%s, %s, textAsBlob(%s));", (forecast['city']['id'], forecast['list'][0]['dt'], forecast.__str__()))

In [16]:
# Query the data
forecast_rows = session.execute("SELECT * FROM forecast_table;")
print(forecast_rows.one()) # <- only one row

Row(city_id=3139081, dt=1691582400, forecast=b"{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1691582400, 'main': {'temp': 286.26, 'feels_like': 285.95, 'temp_min': 283.99, 'temp_max': 286.26, 'pressure': 996, 'sea_level': 996, 'grnd_level': 982, 'humidity': 89, 'temp_kf': 2.27}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'clouds': {'all': 100}, 'wind': {'speed': 4.25, 'deg': 39, 'gust': 9.51}, 'visibility': 10000, 'pop': 0.91, 'rain': {'3h': 0.77}, 'sys': {'pod': 'd'}, 'dt_txt': '2023-08-09 12:00:00'}, {'dt': 1691593200, 'main': {'temp': 285.4, 'feels_like': 285.06, 'temp_min': 284.4, 'temp_max': 285.4, 'pressure': 999, 'sea_level': 999, 'grnd_level': 984, 'humidity': 91, 'temp_kf': 1}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'clouds': {'all': 100}, 'wind': {'speed': 3.07, 'deg': 4, 'gust': 6.57}, 'visibility': 10000, 'pop': 0.51, 'sys': {'pod': 'd'}, 'dt_txt': '2023-08-09 15:00:00